# Scienti API

In [1]:
import requests
import getpass
import pandas as pd
from IPython.display import JSON
import time

## Read `"NIVEL_MAX"` for products of "Apropiación Social del Conocimiento" (ASC)

In [2]:
asc=pd.read_excel('https://docs.google.com/spreadsheets/d/e/2PACX-1vSjpIpwy89MwRwJdXNy0_Z5zCK6XrieZ2_Cf7i1SQTuk4b_hRrNzuCTBgQe3eTOew/pub?output=xlsx')

/usr/local/lib/python3.9/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [4]:
asc[:1]

,COD_RH,COD_PRODUCTO,SGL_CATEGORIA,NIVEL_0,NIVEL_1,NIVEL_2,NIVEL_3,NIVEL_MAX,NÚMERO,CATEGORIAS,ASC
0,150.0,2.0,ART-00,1: Producción bibliográfica,11: Artículo,111: Publicado en revista especializada,NaN,111: Publicado en revista especializada,72886.0,"['ART-00', 'ART-ART', 'ART-GC', 'EPA-EPA', 'PE...",0


In [5]:
dasc=dict(asc[asc['ASC']==1]['NIVEL_MAX'].str.split(': ').str[:2].to_list())

## Get all products ids from Scienti API

In [7]:
k=getpass.getpass('API KEY')

API KEY ········


Get full list of pro

In [8]:
institution='udea'
year=2022
p=requests.get(f"http://apis.colav.co/scienti/info/?apikey={k}&get=ids&institution={institution}&model_year={year}")

In [9]:
p=p.json()

In [10]:
pd.DataFrame(p[0].get('entities'))

,name,ids
0,patent,"[{'COD_RH': '0000204234', 'COD_PATENTE': 1}, {..."
1,product,"[{'COD_RH': '0000000011', 'COD_PRODUCTO': '1'}..."
2,network,"[{'COD_RH': '0000186473', 'COD_RED': 4}, {'COD..."
3,project,"[{'COD_RH': '0000000011', 'COD_PROYECTO': '1'}..."
4,event,"[{'COD_RH': '0000000016', 'COD_EVENTO': 1}, {'..."
5,institution_endorsement,"[{'COD_AVAL_INSTITUCION': 1382}, {'COD_AVAL_IN..."


In [11]:
p=[d for d in p[0].get('entities') if d.get('name')=='product']

In [12]:
len(p[0].get('ids'))

390012

Filter products of ASC 

In [ ]:
ws=[]
i=0
for d in p[0].get('ids'):
    i+=1
    w=requests.get(f"http://apis.colav.co/scienti/product?apikey={k}&model_year=2022&institution=udea&COD_RH={d.get('COD_RH')}&COD_PRODUCTO={d.get('COD_PRODUCTO')}")
    if w.status_code!=200:
        continue
    wj=w.json()
    x=wj.get('COD_TIPO_PRODUCTO')
    if isinstance(x,str) and x in dasc.keys():
        ws.append(wj)
        print(f"{str(len(ws)).zfill(6)}/{str(i).zfill(6)}",end='\r')
        wj['TIPO_PRODUCTO']=dasc.get(x)
        time.sleep(0.01)

In [40]:
len(ws)

16881

In [73]:
import json

In [76]:
fp=open('asc.json','w')
json.dump(ws,fp)
fp.close()

fp=open('asc.json','r')
ws=json.load(fp)
fp.close()

In [78]:
len(ws)

16881

## Export to table

In [ ]:
#https://www.techieclues.com/blogs/how-to-get-variable-name-as-string-in-python
def get_variable_name(variable):
    for name in globals():
        if id(globals()[name]) == id(variable):
            return name
    for name in locals():
        if id(locals()[name]) == id(variable):
            return name
    return None

def get_details(w):
    d=w.get('details')[0]
    if len(d.keys())==1:
        intd=d.get(list(d.keys())[0])
        if intd and isinstance(intd,list):
            newintd={}
            for intk in intd[0].keys():
                if intd[0][intk]:
                    newintd[intk]=intd[0][intk]
    return newintd

general=['COD_RH', 'COD_PRODUCTO', 'COD_TIPO_PRODUCTO','TIPO_PRODUCTO', 'TXT_NME_PROD', 
       'TXT_RESUMEN_PROD', 
        'NRO_ANO_PRESENTA',
        'TXT_WEB_PRODUCTO', 
        'TXT_DOI', 'TXT_INST_PART', 'DTA_EXPOSICION',
       'STA_EVENTO', 'TXT_NME_TITULAR',  
       'NRO_AUTORES_TOTAL', 'NRO_GRUPOS',
       'COD_PROYECTO', 'ANO_INIC', 'ANO_FIM', 'TXT_NOME_PROJETO','details'
       ]
author=['author_TXT_TOTAL_NAMES']
group=['group_NRO_ID_GRUPO', 'group_COD_ID_GRUPO', 'group_NME_GRUPO','group_COD_RH_AREA', 'group_COD_AREA_CONHEC', 
       'group_COD_PROGRAMA', 'group_COD_PROGRAMA_SECUND','group_TXT_PLAN_TRABAJO', 
       'group_TXT_ESTADO_ARTE', 'group_TXT_OBJETIVOS', 'group_TXT_PROD_DESTACADA', 'group_TXT_RETOS', 'group_TXT_VISION',
       'group_knowledge_area']
institution=['institution_COD_INST', 'institution_COD_INST_MACRO', 'institution_NME_INST','institution_SGL_INST', 
             'institution_COD_SECTOR_ECON', 'institution_SGL_PAIS', 'institution_SGL_DEPARTAMENTO']
authors=['authors_COD_RH_OTRO','authors_TXT_TOTAL_NAMES_FILTRO']
project=['project_COD_RH', 'project_COD_PROYECTO', 'project_TXT_NME_PROYECTO', 'project_TXT_NME_PROYECTO_FILTRO', 'project_TXT_OBSERV_PROYECTO', 
         'project_TXT_RESUMEN_PROYECTO', 'project_TPO_PROYECTO', 'project_NRO_ANO_INICIO', 
         'project_NRO_ANO_FIN' 'project_NRO_DURACION', 'project_TPO_ESTADO_PROYECTO']

In [ ]:
Ws=[]
for w in ws:
    W={}
    for k in general:
        W[k]=w.get(k)
    for t in [author,group,institution,institution,project]:
        sub=get_variable_name(t)

        if w.get(sub) and isinstance(w.get(sub),list):
            for kt in t:
                subk=kt.split(f'{sub}_')[-1]
                W[kt]=w.get(sub)[0].get(subk)
                #break
        #if sub=='group' and w.get('group'):
        #        raise Exception('group')
    
    if w.get('details'):
        W['details']=get_details(w)    
    Ws.append(W)
    #break

In [ ]:
df=pd.DataFrame(Ws)
df.to_excel('ASC.xlsx',index=None)

In [ ]:
ls -lh ASC.xlsx

In [ ]:
df.columns

In [ ]:
df.dropna(subset=['author_TXT_TOTAL_NAMES','group_NRO_ID_GRUPO','institution_NME_INST',
                 'project_TXT_NME_PROYECTO','TXT_RESUMEN_PROD']).iloc[0].to_dict()

## Appendix

In [42]:
df=pd.DataFrame(ws)
df.columns

Index(['COD_RH', 'COD_PRODUCTO', 'TXT_NME_PROD', 'TXT_NME_PROD_FILTRO',
       'TXT_RESUMEN_PROD', 'TXT_OBSERV_PROD', 'SGL_IDIOMA',
       'COD_TIPO_PRODUCTO', 'COD_SUBTIPO_PRODUCTO', 'TXT_COMPLEMENTARIO',
       'COD_TIPO_PRODUCTO_MDCN', 'SGL_PAIS', 'NRO_ANO_PRESENTA',
       'NRO_MES_PRESENTA', 'TXT_WEB_PRODUCTO', 'NRO_IND_EXISTENCIA_MDCN',
       'NRO_IND_CALIDAD_MDCN', 'TXT_IND_CALIDAD_MDCN',
       'NRO_IND_VISIBILIDAD_MDCN', 'TPO_AMPERSAN_MDCN',
       'TPO_MEDIO_DIVULGACION', 'STA_RELEVANCIA', 'COD_PRODUCTO_BNR',
       'STA_CERTIFICADO', 'STA_VERIFICACION_AUTO', 'DTA_VERIF_AUTO',
       'STA_VERIFICACION_USUA', 'DTA_VERIF_USUA', 'TXT_FICHA_RESUMEN',
       'DTA_CREACION', 'DTA_ACTUALIZACION', 'STA_VENTANA', 'NRO_PATENTES',
       'NRO_SECRETOS', 'COD_UNO_MDCN', 'COD_DOS_MDCN', 'COD_TRES_MDCN',
       'STA_VENTANA_2', 'TXT_IND_CALIDAD_MDCN_2', 'TIPO_PRODUCTO_MDCN',
       'TIPO_PRODUCTO_MDCN_2', 'TXT_DOI', 'TXT_INST_PART', 'DTA_EXPOSICION',
       'STA_EVENTO', 'TXT_NME_TITULAR'

In [ ]:
general=['COD_RH', 'COD_PRODUCTO', 'COD_TIPO_PRODUCTO','TIPO_PRODUCTO', 'TXT_NME_PROD', 
       'TXT_RESUMEN_PROD', 
        'NRO_ANO_PRESENTA',
        'TXT_WEB_PRODUCTO', 
       'TXT_FICHA_RESUMEN',
        'TXT_DOI', 'TXT_INST_PART', 'DTA_EXPOSICION',
       'STA_EVENTO', 'TXT_NME_TITULAR',  'author', 'product_type',
       'author_others', 'details', 'language',  '
       'NRO_AUTORES_TOTAL', 'NRO_GRUPOS',  'group', 'institution', 'project', 
       'COD_PROYECTO', 'ANO_INIC', 'ANO_FIM', 'TXT_NOME_PROJETO',
       ]

In [49]:
df['author'].iloc[0][0].keys()

dict_keys(['COD_RH', 'NRO_ID_CNPQ', 'TXT_NAMES_RH', 'TXT_PRIM_APELL', 'TXT_SEG_APELL', 'TXT_TOTAL_NAMES', 'TXT_NAMES_RH_FILTRO', 'TXT_PRIM_APELL_FILTRO', 'TXT_SEG_APELL_FILTRO', 'TXT_TOTAL_NAMES_FILTRO', 'TPO_NACIONALIDAD', 'DTA_NACIM', 'TPO_DOCUMENTO_IDENT', 'TPO_SEXO', 'COD_RH_MUN_NACIM', 'COD_MUN_NACIM', 'NRO_DOCUMENTO_IDENT', 'COD_RH_MUN_EXPED_DOC', 'COD_MUN_EXPED_DOC', 'NRO_LIBRETA_MILITAR', 'TPO_LIBRETA_MILITAR', 'NRO_PASAPORTE', 'NRO_VISA', 'TPO_VISA', 'DTA_INI_VIGENCIA_VISA', 'DTA_FIN_VIGENCIA_VISA', 'COD_RH_RELIGION', 'COD_RELIGION', 'SGL_PAIS_NACIM', 'TXT_AFICION', 'TXT_CONGREGA_PERTENECE', 'TXT_CONGREGA_PERTENECIO', 'NRO_CEDULA_EXTRANJ', 'TXT_COMENT_SUGEREN', 'DTA_DEFUNC', 'COD_NIVEL_FORMACION', 'TPO_ESTADO_CIVIL', 'TPO_DIRECCION_USUAL', 'TPO_DOC_ACTUALIZACION', 'TPO_VER_DOC_ACTUALIZACION', 'TXT_CITACION_BIBLIO', 'TXT_CITACION_BIBLIO_FILTRO', 'STA_CERTIFICADO', 'DTA_CERTIFICADO', 'TXT_RESPUESTA', 'TXT_PREGUNTA', 'DTA_CREACION', 'DTA_ACTUALIZACION', 'COD_CERTIFICACION', 'STA_

In [ ]:
author=['author_TXT_TOTAL_NAMES']

In [53]:
df['group'].dropna().iloc[0][0].keys()

dict_keys(['NRO_ID_GRUPO', 'COD_ID_GRUPO', 'NME_GRUPO', 'NME_GRUPO_FILTRO', 'ANO_FORMACAO', 'MES_FORMACAO', 'COD_RH_AREA', 'COD_AREA_CONHEC', 'COD_PROGRAMA', 'COD_PROGRAMA_SECUND', 'TPO_ESTADO_GR', 'DTA_ESTADO_GR', 'TXT_CLASIF', 'DTA_CLASIF', 'DTA_FIN_CLASIF', 'TPO_GRUPO', 'DTA_TPO_GRUPO', 'STA_ELIMINADO', 'TXT_PLAN_TRABAJO', 'TXT_ESTADO_ARTE', 'TXT_OBJETIVOS', 'TXT_PROD_DESTACADA', 'TXT_RETOS', 'TXT_VISION', 'TXT_SUGERENCIAS', 'XML_FICHA_RSM', 'STA_CERTIFICADO', 'DTA_CERTIFICADO', 'DTA_CREACION', 'DTA_ACTUALIZACION', 'COD_CERTIFICACION', 'COD_SESION', 'DTA_INI_SESION', 'TXT_CLASIF_1', 'COD_GRAN_AREA_CONHEC_MED', 'STA_CENTRO', 'TPO_CENTRO', 'STA_SECTOR', 'NRO_INTENTOS', 'STA_ACTIVO', 'COD_ACTIVACION', 'NRO_BLOQUEOS', 'COD_AREA_CONHEC_BK', 'institution', 'knowledge_area'])

In [ ]:
group=['group_NRO_ID_GRUPO', 'group_COD_ID_GRUPO', 'group_NME_GRUPO','group_COD_RH_AREA', 'group_COD_AREA_CONHEC', 
       'group_COD_PROGRAMA', 'group_COD_PROGRAMA_SECUND','group_TXT_PLAN_TRABAJO', 
       'group_TXT_ESTADO_ARTE', 'group_TXT_OBJETIVOS', 'group_TXT_PROD_DESTACADA', 'group_TXT_RETOS', 'group_TXT_VISION', 'group_TXT_SUGERENCIAS']


In [58]:
df['institution'].dropna().iloc[0][0].keys()

dict_keys(['COD_INST', 'COD_INST_MACRO', 'NME_INST', 'NME_INST_FILTRO', 'SGL_INST', 'COD_SECTOR_ECON', 'SGL_PAIS', 'SGL_DEPARTAMENTO', 'COD_RH_MUNICIPIO', 'COD_MUNICIPIO', 'URL_HOME_PAGE', 'TXT_DIRECCION', 'TXT_TELEFONO', 'TXT_FAX', 'TXT_EMAIL', 'TXT_NIT', 'TXT_DIGITO_VERIFICADOR', 'ID_REPRESENTANTE', 'TPO_AVAL', 'TXT_CIUDAD_INST', 'TPO_CARACTER', 'TPO_CONTRIBUYENTE', 'TXT_APARTADO_AEREO', 'STA_EXPORTA', 'COD_NIVEL', 'COD_IES', 'DTA_CONSTITUCION', 'DTA_CREACION', 'DTA_ACTUALIZACION', 'COD_SECTOR_ECON2', 'ID_INSTITUCION', 'STA_TERMINOS_COND', 'DTA_ACEPTA_TERMINOS'])

In [ ]:
institution=['institution_COD_INST', 'institution_COD_INST_MACRO', 'institution_NME_INST','institution_SGL_INST', 'institution_COD_SECTOR_ECON', 'institution_SGL_PAIS', 'institution_SGL_DEPARTAMENTO']

In [59]:
df['author_others'].dropna().iloc[0][0].keys()

dict_keys(['COD_RH_CREA', 'COD_RH_OTRO', 'TXT_NME_RH', 'TXT_PRIM_APELL', 'TXT_SEG_APELL', 'TXT_TOTAL_NAMES_FILTRO', 'SGL_PAIS', 'TPO_NACIONALIDAD', 'DTA_NACIMIENTO', 'TPO_DOC_IDENTIFICACION', 'NRO_DOC_IDENTIFICACION', 'NRO_ID_CNPQ', 'DTA_CREACION', 'DTA_ACTUALIZACION', 'NRO_DOC_FILTRO', 'TXT_TOTAL_NAMES', 'TXT_NME_RH_FILTRO', 'TXT_PRIM_APELL_FILTRO', 'TXT_SEG_APELL_FILTRO', 'COD_RH_REF', 'COD_ORCID', 'AUTOR_ID_SCP'])

In [ ]:
authors=['authors_COD_RH_OTRO','authors_TXT_TOTAL_NAMES_FILTRO']

In [72]:
df['project'].dropna().iloc[0][0].keys()

dict_keys(['COD_RH', 'COD_PROYECTO', 'TXT_NME_PROYECTO', 'TXT_NME_PROYECTO_FILTRO', 'TXT_OBSERV_PROYECTO', 'TXT_RESUMEN_PROYECTO', 'TPO_PROYECTO', 'NRO_MES_INICIO', 'NRO_ANO_INICIO', 'NRO_MES_FIN', 'NRO_ANO_FIN', 'NRO_MES_PROYEC', 'NRO_ANO_PROYEC', 'NRO_DURACION', 'TPO_ESTADO_PROYECTO', 'NRO_GRADUACION', 'NRO_ESPECIALIZACION', 'NRO_MAESTRIA_ACAD', 'NRO_MAESTRIA_PROF', 'NRO_DOCTORADO', 'COD_PROYECTO_BNR', 'STA_CERTIFICADO', 'STA_VERIFICACION_AUTO', 'DTA_VERIF_AUTO', 'STA_VERIFICACION_USUA', 'DTA_VERIF_USUA', 'TXT_FICHA_RESUMEN', 'DTA_CREACION', 'DTA_ACTUALIZACION', 'TXT_URL', 'COD_PROYECTO_SNCTI', 'TPO_FINANCIACION', 'TPO_FTE_FINAN', 'TPO_AMB_FINAN', 'STA_AVAL_INST', 'DTA_AVAL_INST', 'TPO_AVAL_INST', 'COD_INST_AVALA', 'ID_USUARIO_AVALA', 'DTA_ACTUALIZACION_CVLAC', 'CAT_MED', 'SGL_CATEGORIA'])

In [ ]:
project=['project_COD_RH', 'project_COD_PROYECTO', 'project_TXT_NME_PROYECTO', 'project_TXT_NME_PROYECTO_FILTRO', 'project_TXT_OBSERV_PROYECTO', 
         'project_TXT_RESUMEN_PROYECTO', 'project_TPO_PROYECTO', 'project_NRO_ANO_INICIO', 
         'project_NRO_ANO_FIN' 'project_NRO_DURACION', 'project_TPO_ESTADO_PROYECTO']

In [67]:
df['details'].dropna()

0        [{'technical': [{'COD_RH': '0000000011', 'COD_...
1        [{'course': [{'COD_RH': '0000000011', 'COD_PRO...
2        [{'course': [{'COD_RH': '0000000011', 'COD_PRO...
3        [{'technical': [{'COD_RH': '0000000011', 'COD_...
4        [{'course': [{'COD_RH': '0000000011', 'COD_PRO...
                               ...                        
16876    [{'course': [{'COD_RH': '0000771899', 'COD_PRO...
16877    [{'course': [{'COD_RH': '0000771899', 'COD_PRO...
16878    [{'course': [{'COD_RH': '0000771899', 'COD_PRO...
16879    [{'course': [{'COD_RH': '0000771899', 'COD_PRO...
16880    [{'course': [{'COD_RH': '0000771899', 'COD_PRO...
Name: details, Length: 16426, dtype: object

In [16]:
f"http://apis.colav.co/scienti/product?apikey={k}&model_year=2022&institution=udea&COD_RH={d.get('COD_RH')}&COD_PRODUCTO={d.get('COD_PRODUCTO')}"

'http://apis.colav.co/scienti/product?apikey=colavudea&model_year=2022&institution=udea&COD_RH=0000000011&COD_PRODUCTO=3'

In [12]:
x=w.json().get('COD_TIPO_PRODUCTO')
if isinstance(x,str) and x in dasc.keys():
    pass

{'COD_RH': '0000140023',
 'COD_PRODUCTO': '11',
 'TXT_NME_PROD': 'Pervivencia del Derecho español durante el siglo XIX y proceso de codificación civil en Colombia',
 'TXT_NME_PROD_FILTRO': 'PERVIVENCIA DEL DERECHO ESPANOL DURANTE EL SIGLO XIX Y PROCESO DE CODIFICACION CIVIL EN COLOMBIA',
 'TXT_RESUMEN_PROD': None,
 'TXT_OBSERV_PROD': None,
 'SGL_IDIOMA': 'ES',
 'COD_TIPO_PRODUCTO': '121',
 'COD_SUBTIPO_PRODUCTO': None,
 'TXT_COMPLEMENTARIO': None,
 'COD_TIPO_PRODUCTO_MDCN': '119',
 'SGL_PAIS': 'CHL',
 'NRO_ANO_PRESENTA': 1991,
 'NRO_MES_PRESENTA': 1,
 'TXT_WEB_PRODUCTO': None,
 'NRO_IND_EXISTENCIA_MDCN': 1,
 'NRO_IND_CALIDAD_MDCN': None,
 'TXT_IND_CALIDAD_MDCN': '0',
 'NRO_IND_VISIBILIDAD_MDCN': None,
 'TPO_AMPERSAN_MDCN': None,
 'TPO_MEDIO_DIVULGACION': 'I',
 'STA_RELEVANCIA': 'F',
 'COD_PRODUCTO_BNR': None,
 'STA_CERTIFICADO': None,
 'STA_VERIFICACION_AUTO': None,
 'DTA_VERIF_AUTO': None,
 'STA_VERIFICACION_USUA': None,
 'DTA_VERIF_USUA': None,
 'TXT_FICHA_RESUMEN': None,
 'DTA_CREAC